<a href="https://colab.research.google.com/github/Santosdevbjj/generosLiterarios/blob/main/GenerosLivros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import mysql.connector

# Classe para representar um Livro
# O construtor __init__ inicializa os atributos de um objeto Livro.
class Livro:
    def __init__(self, titulo, autor, ano_publicacao, genero_literario, best_seller, pais, idioma):
        self.titulo = titulo
        self.autor = autor
        self.ano_publicacao = ano_publicacao
        self.genero_literario = genero_literario
        self.best_seller = best_seller
        self.pais = pais
        self.idioma = idioma

    # Método __str__ para uma representação amigável do objeto.
    def __str__(self):
        return (f"Título: {self.titulo} | Autor: {self.autor} | Ano: {self.ano_publicacao} | "
                f"Gênero: {self.genero_literario} | Best Seller: {'Sim' if self.best_seller else 'Não'} | "
                f"País: {self.pais} | Idioma: {self.idioma}")

# Classe que gerencia todas as operações do banco de dados (CRUD)
class GerenciadorLivros:
    def __init__(self, host, user, password, database):
        self.host = host
        self.user = user
        self.password = password
        self.database = database
        self.conexao = None
        self.cursor = None

    # Método para conectar ao banco de dados
    def conectar(self):
        try:
            self.conexao = mysql.connector.connect(
                host=self.host,
                user=self.user,
                password=self.password,
                database=self.database
            )
            self.cursor = self.conexao.cursor()
            print("Conexão com o banco de dados estabelecida!")
        except mysql.connector.Error as erro:
            print(f"Erro ao conectar ao MySQL: {erro}")
            self.conexao = None
            self.cursor = None

    # Método para desconectar do banco de dados
    def desconectar(self):
        if self.conexao and self.conexao.is_connected():
            self.cursor.close()
            self.conexao.close()
            print("Conexão com o banco de dados fechada.")

    # Método para criar a tabela 'livros' se ela não existir
    def criar_tabela(self):
        if not self.conexao or not self.conexao.is_connected():
            print("Não há conexão ativa com o banco de dados.")
            return

        sql = """
        CREATE TABLE IF NOT EXISTS livros (
            id INT AUTO_INCREMENT PRIMARY KEY,
            titulo VARCHAR(255) NOT NULL,
            autor VARCHAR(255) NOT NULL,
            ano_publicacao INT,
            genero_literario VARCHAR(100),
            best_seller BOOLEAN,
            pais VARCHAR(100),
            idioma VARCHAR(50)
        )
        """
        try:
            self.cursor.execute(sql)
            self.conexao.commit()
            print("Tabela 'livros' criada ou já existente.")
        except mysql.connector.Error as erro:
            print(f"Erro ao criar tabela: {erro}")
            self.conexao.rollback()

    # Método para inserir um novo livro
    def inserir_livro(self, livro):
        if not self.conexao or not self.conexao.is_connected():
            print("Não há conexão ativa com o banco de dados.")
            return

        sql = """
        INSERT INTO livros (titulo, autor, ano_publicacao, genero_literario, best_seller, pais, idioma)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        valores = (livro.titulo, livro.autor, livro.ano_publicacao, livro.genero_literario,
                   livro.best_seller, livro.pais, livro.idioma)
        try:
            self.cursor.execute(sql, valores)
            self.conexao.commit()
            print(f"Livro '{livro.titulo}' inserido com sucesso.")
            return self.cursor.lastrowid # Retorna o ID do livro inserido
        except mysql.connector.Error as erro:
            print(f"Erro ao inserir livro: {erro}")
            self.conexao.rollback()
            return None

    # Método para ler todos os livros
    def ler_livros(self):
        if not self.conexao or not self.conexao.is_connected():
            print("Não há conexão ativa com o banco de dados.")
            return []

        sql = "SELECT * FROM livros"
        try:
            self.cursor.execute(sql)
            resultados = self.cursor.fetchall()
            livros = []
            for row in resultados:
                # Cria objetos Livro a partir dos resultados do banco
                livro = Livro(row[1], row[2], row[3], row[4], row[5], row[6], row[7])
                livros.append(livro)
            return livros
        except mysql.connector.Error as erro:
            print(f"Erro ao ler livros: {erro}")
            return []

    # Método para buscar livros por critério
    def buscar_livros(self, criterio, valor):
        if not self.conexao or not self.conexao.is_connected():
            print("Não há conexão ativa com o banco de dados.")
            return []

        # Mapeia o critério de busca para a coluna do banco de dados
        coluna = {
            'titulo': 'titulo',
            'autor': 'autor',
            'ano': 'ano_publicacao',
            'genero': 'genero_literario',
            'best_seller': 'best_seller',
            'pais': 'pais',
            'idioma': 'idioma'
        }.get(criterio.lower())

        if not coluna:
            print("Critério de busca inválido.")
            return []

        sql = f"SELECT * FROM livros WHERE {coluna} LIKE %s"
        try:
            self.cursor.execute(sql, (f"%{valor}%",))
            resultados = self.cursor.fetchall()
            livros = []
            for row in resultados:
                livro = Livro(row[1], row[2], row[3], row[4], row[5], row[6], row[7])
                livros.append(livro)
            return livros
        except mysql.connector.Error as erro:
            print(f"Erro ao buscar livros: {erro}")
            return []


    # Método para atualizar um livro
    def atualizar_livro(self, id_livro, novo_livro):
        if not self.conexao or not self.conexao.is_connected():
            print("Não há conexão ativa com o banco de dados.")
            return False

        sql = """
        UPDATE livros
        SET titulo = %s, autor = %s, ano_publicacao = %s, genero_literario = %s,
            best_seller = %s, pais = %s, idioma = %s
        WHERE id = %s
        """
        valores = (novo_livro.titulo, novo_livro.autor, novo_livro.ano_publicacao,
                   novo_livro.genero_literario, novo_livro.best_seller,
                   novo_livro.pais, novo_livro.idioma, id_livro)
        try:
            self.cursor.execute(sql, valores)
            self.conexao.commit()
            print(f"Livro com ID {id_livro} atualizado com sucesso.")
            return True
        except mysql.connector.Error as erro:
            print(f"Erro ao atualizar livro: {erro}")
            self.conexao.rollback()
            return False

    # Método para deletar um livro
    def deletar_livro(self, id_livro):
        if not self.conexao or not self.conexao.is_connected():
            print("Não há conexão ativa com o banco de dados.")
            return False

        sql = "DELETE FROM livros WHERE id = %s"
        try:
            self.cursor.execute(sql, (id_livro,))
            self.conexao.commit()
            print(f"Livro com ID {id_livro} deletado com sucesso.")
            return True
        except mysql.connector.Error as erro:
            print(f"Erro ao deletar livro: {erro}")
            self.conexao.rollback()
            return False

# Função para exibir o menu principal
def exibir_menu():
    print("\n--- Sistema de Gerenciamento de Livros ---")
    print("1. Adicionar novo livro")
    print("2. Listar todos os livros")
    print("3. Buscar livros por critério")
    print("4. Atualizar informações de um livro")
    print("5. Deletar um livro")
    print("6. Sair")

# Função principal que roda o sistema
def main():
    # Cria uma instância do nosso gerenciador
    gerenciador = GerenciadorLivros(
        host="localhost",
        user="usuario_crud",
        password="senha123",
        database="mysql" # Usaremos o banco de dados padrão 'mysql' para simplificar
    )

    # Conecta e cria a tabela
    gerenciador.conectar()
    if gerenciador.conexao is None:
        print("Não foi possível conectar ao banco de dados. Encerrando o programa.")
        return

    gerenciador.criar_tabela()

    while True:
        exibir_menu()
        escolha = input("Escolha uma opção: ")

        # Opção 1: Adicionar um novo livro
        if escolha == '1':
            titulo = input("Título: ")
            autor = input("Autor: ")
            try:
                ano = int(input("Ano de Publicação: "))
            except ValueError:
                print("Ano de Publicação inválido. Por favor, insira um número.")
                continue
            genero = input("Gênero Literário: ")
            best_seller_str = input("É um Best Seller? (s/n): ").lower()
            best_seller = best_seller_str == 's'
            pais = input("País: ")
            idioma = input("Idioma: ")
            novo_livro = Livro(titulo, autor, ano, genero, best_seller, pais, idioma)
            gerenciador.inserir_livro(novo_livro)

        # Opção 2: Listar todos os livros
        elif escolha == '2':
            print("\n--- Livros Cadastrados ---")
            livros = gerenciador.ler_livros()
            if livros:
                for livro in livros:
                    print(livro)
            else:
                print("Nenhum livro cadastrado.")

        # Opção 3: Buscar livros por critério
        elif escolha == '3':
            print("\n--- Opções de Busca ---")
            print("1. Título\n2. Autor\n3. Ano de Publicação\n4. Gênero Literário\n5. Best Seller\n6. País\n7. Idioma")
            criterio_escolha = input("Escolha o critério de busca (1-7): ")

            criterios = {
                '1': 'titulo', '2': 'autor', '3': 'ano', '4': 'genero',
                '5': 'best_seller', '6': 'pais', '7': 'idioma'
            }

            criterio = criterios.get(criterio_escolha)
            if criterio:
                valor = input(f"Digite o valor para {criterio}: ")
                livros_encontrados = gerenciador.buscar_livros(criterio, valor)
                if livros_encontrados:
                    print("\n--- Livros Encontrados ---")
                    for livro in livros_encontrados:
                        print(livro)
                else:
                    print("Nenhum livro encontrado com este critério.")
            else:
                print("Opção de critério inválida.")

        # Opção 4: Atualizar informações de um livro
        elif escolha == '4':
            try:
                id_para_atualizar = int(input("Digite o ID do livro a ser atualizado: "))
                # Seria ideal buscar o livro primeiro para exibir as informações atuais
                # Mas para simplificar, pedimos todos os dados novamente
                titulo = input("Novo Título: ")
                autor = input("Novo Autor: ")
                ano = int(input("Novo Ano de Publicação: "))
                genero = input("Novo Gênero Literário: ")
                best_seller_str = input("É um Best Seller? (s/n): ").lower()
                best_seller = best_seller_str == 's'
                pais = input("Novo País: ")
                idioma = input("Novo Idioma: ")
                novo_livro_info = Livro(titulo, autor, ano, genero, best_seller, pais, idioma)
                gerenciador.atualizar_livro(id_para_atualizar, novo_livro_info)
            except ValueError:
                print("ID ou Ano de Publicação inválido. Por favor, insira um número.")


        # Opção 5: Deletar um livro
        elif escolha == '5':
            try:
                id_para_deletar = int(input("Digite o ID do livro a ser deletado: "))
                gerenciador.deletar_livro(id_para_deletar)
            except ValueError:
                print("ID inválido. Por favor, insira um número.")

        # Opção 6: Sair
        elif escolha == '6':
            print("Saindo do sistema.")
            break

        else:
            print("Opção inválida. Tente novamente.")

    # Garante que a conexão seja fechada ao sair
    gerenciador.desconectar()

# Executa a função principal se o script for rodado diretamente
if __name__ == "__main__":
    main()

In [ ]:
# Instala o MySQL server e o cliente
# `-y` para confirmar automaticamente a instalação
!apt-get install -y -qq mysql-server &> /dev/null
print("MySQL server instalado com sucesso.")

# Inicia o serviço do MySQL
!service mysql start
print("MySQL server iniciado.")

# Instala a biblioteca Python para conectar ao MySQL
!pip install -q mysql-connector-python
print("Biblioteca 'mysql-connector-python' instalada.")

# Define um usuário e senha para o banco de dados
# O usuário 'root' não tem senha por padrão no Colab, o que é inseguro em produção,
# mas ideal para este ambiente didático.
!mysql -e "CREATE USER 'usuario_crud'@'localhost' IDENTIFIED BY 'senha123';"
!mysql -e "GRANT ALL PRIVILEGES ON *.* TO 'usuario_crud'@'localhost' WITH GRANT OPTION;"
!mysql -e "FLUSH PRIVILEGES;"

print("Usuário 'usuario_crud' criado e privilégios concedidos.")